In [3]:
!pip install yfinance

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Created wheel for peewee: filename=peewee-3.17.8-py3-none-any.whl size=139017 sha256=7ab9db7158a1bd5e989bbf2762ed86b67f5e50b685366fd7cb7a7a81ebee6144
  Stored in directory: /root/.cache/pip/wheels/75/79/e5/8838db0594cc6c587142fd2563356392ade6255c5930411069
Successfully built peewee
  Attempting uninstall: requests
    Found existing installation: requests 2.30.0
    Uninstalling requests-2.30.0:
      Successfully uninstalled requests-2.30.0


실습N #1: 엔비디아의 주가 데이터를 가져와서 분석하는 어플리케이션을 작성하시오.

In [5]:
import os
import openai
import pandas as pd
import yfinance as yf

openai.api_key = os.getenv("OPENAI_API_KEY")

def analyze_nvda_stock(start_date, end_date):
    """
    엔비디아 주가 데이터를 가져와서 분석 정보를 제공합니다.

    Args:
        start_date (str): 시작 날짜 (YYYYMMDD 형식)
        end_date (str): 종료 날짜 (YYYYMMDD 형식)

    Returns:
        str: 분석 결과 문자열
    """

    try:
        # 엔비디아 티커
        ticker = "NVDA"

        # 주가 데이터 가져오기
        df = yf.download(ticker, start=start_date, end=end_date)
        print(df)

        df_string = df.to_string()
        
        prompt = f"""
                다음은 {ticker} 주식의 {start_date}부터 {end_date}까지의 주가 데이터입니다.
        {df_string}
        이 데이터를 분석하고 주요 트렌드와 인사이트를 제공하십시오.
        """
        
        response = openai.chat.completions.create(  # ChatCompletion 사용
            model="gpt-4o",  # 또는 다른 적절한 모델
            messages=[
                {"role": "system", "content": "주식 분석 전문가"},  # 시스템 역할 설정 (선택적)
                {"role": "user", "content": prompt}
            ],
            max_tokens=1000,
            n=1,
            stop=None,
            temperature=0
        )
        
        analysis = response.choices[0].message.content

        return analysis

    except Exception as e:
        return f"분석 중 오류가 발생했습니다: {e}"


# ChatGPT 연동 예시 (가상)
user_query = "2023년 엔비디아 주가 분석해줘"
start_date, end_date = "2023-01-01", "2023-12-31" # 날짜 추출 로직 필요

analysis = analyze_nvda_stock(start_date, end_date)
print(f"사용자 질문: {user_query}\n{analysis}")

[*********************100%***********************]  1 of 1 completed


Price           Close       High        Low       Open     Volume
Ticker           NVDA       NVDA       NVDA       NVDA       NVDA
Date                                                             
2023-01-03  14.304595  14.985101  14.085754  14.840206  401277000
2023-01-04  14.738280  14.842205  14.230650  14.556413  431324000
2023-01-05  14.254632  14.553414  14.137716  14.480468  389168000
2023-01-06  14.848200  14.999090  14.023800  14.463479  405044000
2023-01-09  15.616640  16.044328  15.129994  15.272890  504231000
...               ...        ...        ...        ...        ...
2023-12-22  48.816162  49.369003  48.453262  49.181057  252507000
2023-12-26  49.265030  49.585938  48.946120  48.954117  244200000
2023-12-27  49.402988  49.665914  49.071082  49.496964  233648000
2023-12-28  49.507965  49.869862  49.397996  49.628933  246587000
2023-12-29  49.507965  49.982833  48.737184  49.798883  389293000

[250 rows x 5 columns]
사용자 질문: 2023년 엔비디아 주가 분석해줘
이 NVDA 주식 데이터는 2023년 1월 1

실습N #2: 사용자가 입력한 지역과 질문에 맞는 지역 음식점을 검색해서 추천하는 어플리케이션을 작성하시오.

In [2]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40716 sha256=10acd3f93562fb81d6d4016e94c5b7c76ba4c014f327a75e482671939db9ae90
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [9]:
import os
import googlemaps
import openai

GOOGLE_PLACE_API="https://places.googleapis.com/v1/places/GyuEmsRBfy61i59si0?fields=addressComponents&key="
GOOGLE_PLACE_API_KEY=os.getenv("GOOGLE_PLACE_API_KEY")

gmaps = googlemaps.Client(key=GOOGLE_PLACE_API_KEY)
openai.api_key = os.getenv("OPENAI_API_KEY")

def extract_location_and_query(user_input):
    """사용자 입력에서 위치와 질문을 추출합니다."""
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "사용자 입력에서 위치와 질문을 추출해주세요. 위치는 최대한 구체적으로 추출합니다. \
            예시: \
            입력: 서울 강남구에서 맛집 추천해줘 \
            출력: {'location': '서울 강남구', 'query': '맛집'} \
            입력: 서울에서 파스타 맛집 추천 \
            출력: {'location': '서울', 'query': '파스타 맛집'} \
            입력: 신촌역 근처 이탈리안 레스토랑 \
            출력: {'location': '신촌역', 'query': '이탈리안 레스토랑'}"},
            {"role": "user", "content": user_input}
        ],
        temperature=0
    )

    extracted_info = response.choices[0].message.content
    # 문자열을 딕셔너리로 변환 (안전한 eval 대신 ast.literal_eval 사용)
    import ast
    try:
        extracted_info = ast.literal_eval(extracted_info)
        return extracted_info['location'], extracted_info['query']
    except (SyntaxError, ValueError):
        print("위치 및 질문 추출에 실패했습니다.")
        return None, None

def recommend_restaurants(location, query):
    """
    사용자 위치와 질문에 따라 음식점을 추천합니다.
    """

    # 1. Google Places API로 장소 검색
    places_result = gmaps.places(query=query, location=location)

    # 2. 검색 결과에서 필요한 정보 추출
    places = []
    if places_result['results']:
        for place in places_result['results']:
            places.append({
                'name': place['name'],
                'address': place['formatted_address'],
                'rating': place.get('rating', 0),  # rating 없을 경우 0으로 설정
            })

    # 3. OpenAI API를 사용하여 추천 메시지 생성
    if places:
        prompt = f"사용자는 '{location}'에서 '{query}'를 검색했습니다. 다음 음식점들을 고려하여 추천해주세요:\n"
        for place in places:
            prompt += f"- {place['name']} ({place['address']}, 평점: {place['rating']})\n"

        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "음식점 추천 전문가"},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )
        recommendations = response.choices[0].message.content
    else:
        recommendations = "검색 결과가 없습니다."

    return recommendations

user_input = input("음식점을 추천받고 싶은 장소와 종류를 한 문장으로 입력하세요 (예: 서울 강남구에서 맛집 추천해줘): ")
location, query = extract_location_and_query(user_input)

if location and query:
    recommendations = recommend_restaurants(location, query)
    print("\n추천 결과:\n", recommendations)


추천 결과:
 서울 강남구에서 맛집을 찾고 계신다면, 강남구와 가까운 지역에 위치한 음식점 중에서 높은 평점을 받은 곳을 추천드리겠습니다. 다음은 추천드리는 음식점 목록입니다:

1. **Taeyang Gopchang Jamsil Bangi-dong** (평점: 5)  
   - 위치: 서울 송파구 방이동
   - 설명: 고급스러운 곱창 요리를 제공하는 곳으로, 높은 평점을 자랑합니다.

2. **대팔이네** (평점: 4.8)  
   - 위치: 서울 강동구 천호동
   - 설명: 다양한 한식 요리를 맛볼 수 있는 곳으로, 맛과 서비스 모두 뛰어납니다.

3. **강동AGU** (평점: 4.7)  
   - 위치: 서울 강동구 암사동
   - 설명: 신선한 재료로 만든 요리를 제공하며, 고객 만족도가 높은 곳입니다.

4. **Ricco Oven Pizza Hanam Misa Main** (평점: 4.8)  
   - 위치: 경기도 하남시
   - 설명: 정통 이탈리안 피자를 맛볼 수 있는 곳으로, 피자 애호가들에게 추천합니다.

이 음식점들은 강남구와 비교적 가까운 위치에 있으며, 높은 평점을 받은 곳들입니다. 각 음식점의 특색 있는 메뉴를 즐기며 맛있는 식사를 경험해 보세요!
